In [303]:
import os
import pandas as pd
import numpy as np
import urllib
import plotly.graph_objects as go
import calendar
from scipy import stats

In [304]:
path = r'C:\Users\student\Documents\Alistair CoAgMET Projects\Pythonstuff\PrecipComparison\snowfall_comp'
os.chdir(path)

In [305]:
csv = path + r'\csv.csv'
f = open(csv,'w+')

In [306]:
urllib.request.urlretrieve(
    'http://coagmet.colostate.edu/cgi-bin/web_services.pl?type=hourly&sids=rfd01&sdate=1992-06-04' + 
    '&edate=2018-12-31&elems=pp',filename=csv)
pp_df = pd.read_csv(csv)

In [307]:
pp_df = pp_df.reset_index()

In [308]:
pp_df.columns = ['station','datetime','precip']
pp_df['datetime'] = pd.to_datetime(pp_df.datetime)

In [309]:
r = pd.date_range(start=pp_df.datetime.min(),end=pp_df.datetime.max(),freq='H')
pp_df = pp_df.set_index('datetime').reindex(r).fillna(np.nan).rename_axis('datetime').reset_index()

In [310]:
pp_df['datetime'] = pd.to_datetime(pp_df.datetime)
pp_df = pp_df.set_index(pp_df.datetime)
pp_df = pp_df.resample(rule='24H',label='right',base=17).agg(pd.Series.sum, skipna=False)

In [311]:
pp_df['date'] = pp_df.index.date
pp_df['date'] = pd.to_datetime(pp_df.date)
pp_df = pp_df.set_index(pp_df.date)


In [312]:
pp_df.precip = pp_df.precip / 25.4

In [313]:
urllib.request.urlretrieve(
    'http://coagmet.colostate.edu/cgi-bin/web_services.pl?type=daily&sids=rfd01&sdate=1992-06-04' + 
    '&edate=2018-12-31&elems=tave,tmax,tmin,vp,sr',filename=csv)
temp_rh_df = pd.read_csv(csv)

In [314]:
temp_rh_df = temp_rh_df.reset_index()
temp_rh_df.columns = ['station','date','tave','tmax','tmin','vp','sr']
temp_rh_df['date'] = pd.to_datetime(temp_rh_df.date)

In [315]:
r = pd.date_range(start=temp_rh_df.date.min(),end=temp_rh_df.date.max(),freq='D')
temp_rh_df = temp_rh_df.set_index('date').reindex(r).fillna(np.nan).rename_axis('date').reset_index()
temp_rh_df['date'] = pd.to_datetime(temp_rh_df.date)
temp_rh_df = temp_rh_df.set_index(temp_rh_df.date)

In [316]:
coag_df = pp_df.merge(temp_rh_df,left_on=pp_df.index,right_on=temp_rh_df.index)

In [317]:
stuff = ['date_x','date_y']
coag_df = coag_df.drop(stuff,axis=1)

In [318]:
coag_df.columns = ['date','precip','station','tave','tmax','tmin','vp','sr']

In [319]:
urllib.request.urlretrieve(
    'http://data.rcc-acis.org/StnData?sid=057167&sdate=1992-06-04&edate=2018-12-31' + 
    '&elems=pcpn,snow,snwd&output=csv',filename=csv)
coop_df = pd.read_csv(csv)

In [320]:
coop_df = coop_df.reset_index()
coop_df.columns = ['date','coop_precip','snow','snowdepth']

In [321]:
coop_df['date'] = pd.to_datetime(coop_df.date)
coop_df = coop_df.set_index(coop_df.date)

In [322]:
coag_df['date'] = pd.to_datetime(coag_df.date)
coop_df['date'] = pd.to_datetime(coop_df.date)
df = coag_df.merge(coop_df, left_on=coag_df.date,right_on=coop_df.date)
df

,key_0,date_x,precip,station,tave,tmax,tmin,vp,sr,date_y,coop_precip,snow,snowdepth
0,1992-06-04,1992-06-04,NaN,rfd01,17.350,24.590,11.610,1.496,17.500,1992-06-04,T,0.0,0
1,1992-06-05,1992-06-05,0.090000,rfd01,18.990,29.640,12.160,1.587,22.810,1992-06-05,0.12,0.0,0
2,1992-06-06,1992-06-06,0.020000,rfd01,16.110,24.600,8.380,1.267,23.390,1992-06-06,0.03,0.0,0
3,1992-06-07,1992-06-07,0.090000,rfd01,19.690,29.420,11.550,1.466,21.150,1992-06-07,0.12,0.0,0
4,1992-06-08,1992-06-08,0.000000,rfd01,16.570,24.680,12.350,1.475,18.460,1992-06-08,0.00,0.0,0
5,1992-06-09,1992-06-09,0.560000,rfd01,15.200,20.290,10.780,1.497,15.700,1992-06-09,1.13,0.0,0
6,1992-06-10,1992-06-10,0.090000,rfd01,17.720,27.020,7.900,1.415,29.330,1992-06-10,0.00,0.0,0
7,1992-06-11,1992-06-11,0.030000,rfd01,17.630,25.510,11.690,1.697,23.930,1992-06-11,T,0.0,0
8,1992-06-12,1992-06-12,0.040000,rfd01,19.370,27.080,10.790,1.796,29.260,1992-06-12,0.80,0.0,0
9,1992-06-13,1992-06-13,0.020000,rfd01,21.520,31.490,13.860,1.722,29.540,1992-06-13,0.00,0.0,0


In [323]:
stuff = ['date_x','date_y']
df = df.drop(stuff, axis=1)
df.columns = ['date','coag_precip','station','tave','tmax','tmin','vp','sr','coop_precip',
             'snow','snowdepth']

In [324]:
df.to_csv(path + r'\rfd01_precip_snow_coopcomp.csv')

In [325]:
df = df.replace('M',np.nan,regex=True)
df = df.replace('T',0.005,regex=True)

df.coop_precip = df.coop_precip.astype(float)
df.snow = df.snow.astype(float)

df['snowfall'] = np.nan
df['tot_precip'] = np.nan

n = 0
snowfall = 0 
precip = 0
for index,rows in df.iterrows():
    if df.snow[n] > 0:
        df['snowfall'][n] = 1
    n = n + 1
d = 0
for index,rows in df.iterrows():
    if df.coop_precip[d] > 0:
        df['tot_precip'] = 1
    d = d + 1
df

C:\Users\student\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,date,coag_precip,station,tave,tmax,tmin,vp,sr,coop_precip,snow,snowdepth,snowfall,tot_precip
0,1992-06-04,NaN,rfd01,17.350,24.590,11.610,1.496,17.500,0.005,0.0,0,NaN,1
1,1992-06-05,0.090000,rfd01,18.990,29.640,12.160,1.587,22.810,0.120,0.0,0,NaN,1
2,1992-06-06,0.020000,rfd01,16.110,24.600,8.380,1.267,23.390,0.030,0.0,0,NaN,1
3,1992-06-07,0.090000,rfd01,19.690,29.420,11.550,1.466,21.150,0.120,0.0,0,NaN,1
4,1992-06-08,0.000000,rfd01,16.570,24.680,12.350,1.475,18.460,0.000,0.0,0,NaN,1
5,1992-06-09,0.560000,rfd01,15.200,20.290,10.780,1.497,15.700,1.130,0.0,0,NaN,1
6,1992-06-10,0.090000,rfd01,17.720,27.020,7.900,1.415,29.330,0.000,0.0,0,NaN,1
7,1992-06-11,0.030000,rfd01,17.630,25.510,11.690,1.697,23.930,0.005,0.0,0,NaN,1
8,1992-06-12,0.040000,rfd01,19.370,27.080,10.790,1.796,29.260,0.800,0.0,0,NaN,1
9,1992-06-13,0.020000,rfd01,21.520,31.490,13.860,1.722,29.540,0.000,0.0,0,NaN,1


In [326]:
df['month'] = df.date.dt.month
df['year'] = df.date.dt.year

In [327]:
monthly_comp = df.groupby(['month','year']).sum().reset_index()
cols = ['month','year','coag_precip','coop_precip','snow','snowfall']
monthly_comp_precip = monthly_comp[cols]

In [328]:
df.dtypes

date           datetime64[ns]
coag_precip           float64
station                object
tave                  float64
tmax                  float64
tmin                  float64
vp                    float64
sr                    float64
coop_precip           float64
snow                  float64
snowdepth              object
snowfall              float64
tot_precip              int64
month                   int64
year                    int64
dtype: object

In [329]:
it_snowed = monthly_comp_precip.snow > 0
monthly_comp_precip = monthly_comp_precip[it_snowed]
monthly_comp_precip['month'] = monthly_comp_precip['month'].apply(lambda x:calendar.month_abbr[x])
monthly_comp_precip['year'] = monthly_comp_precip.year.astype(str)
monthly_comp_precip['ym'] = (monthly_comp_precip['month']) + '-' + (monthly_comp_precip['year'])
monthly_comp_precip['snowfall'] = monthly_comp_precip.snowfall.astype(str)

In [330]:
monthly_comp_precip['rez'] = monthly_comp_precip.coag_precip - monthly_comp_precip.coop_precip

In [331]:
monthly_comp_precip = monthly_comp_precip.round(decimals=2)
precip_comp_fig = go.Figure(data=[go.Scatter()],
                           layout=go.Layout(title=go.layout.Title(text='Monthly Precip Differece vs Monthly Snowfall'),
                                           xaxis=go.layout.XAxis(title=go.layout.xaxis.Title(text='Snowfall (inches)')),
                                           yaxis=go.layout.YAxis(title=go.layout.yaxis.Title(text='Precip Difference (Inches)'))))

precip_comp_fig.add_trace(
    go.Scatter(x=monthly_comp_precip.snow,y=monthly_comp_precip.rez,
              text=monthly_comp_precip.ym + '<br \>'
              + 'Snowfall Events: ' + monthly_comp_precip.snowfall,mode='markers'))

In [332]:
trend = stats.linregress(monthly_comp_precip.snow,monthly_comp_precip.rez)

In [333]:
trend_start = 0*trend.slope+trend.intercept
trend_end = 23.51*trend.slope+trend.intercept
precip_comp_fig.add_trace(go.Scatter(x=[0,30],y=[trend_start,trend_end],mode='lines',name='Trend'))
precip_comp_fig.show()

In [ ]:
snowfall_events_comp_fig = go.Figure(data=[go.Scatter()],
                           layout=go.Layout(title=go.layout.Title(text='Monthly Precip Differece vs Monthly Snowfall'),
                                           xaxis=go.layout.XAxis(title=go.layout.xaxis.Title(text='Snowfall Events (# of Occurences)')),
                                           yaxis=go.layout.YAxis(title=go.layout.yaxis.Title(text='Precip Difference (Inches)'))))
snowfall_events_comp_fi.add_trace(
    go.Scatter(x=monthly_comp_precipy.snowfall,y=monthly_comp_precip.rez,
              text=monthly_comp_precip.ym + '<b \>'))
